In [1]:
# sys.path를 수정하여 모듈 경로를 설정
import sys
import os

# 프로젝트 루트 경로 추가
project_root = os.path.abspath("..")
sys.path.append(project_root)
import module

In [2]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# AppleGothic
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
df = pd.read_csv('../data/unique_apt_group_xy_v3.csv')

In [5]:
df['좌표X'].isnull().sum()

170

In [8]:
보완할대상keymap = pd.read_csv('../data/x_y_key_map.csv')

In [9]:
보완할대상keymap.head(1)

,key,search_key,좌표X,좌표Y,아파트명,search_key1,search_key2,구,동,시,번지,본번,부번,도로명
0,강남구개포동개포로 264개포래미안포레스트,개포로 264 개포래미안포레스트,127.054031,37.47848,개포래미안포레스트,서울특별시 강남구 개포동 개포로 264,서울특별시 강남구 개포동 1282,강남구,개포동,서울특별시,1282,1282.0,0.0,개포로 264


In [10]:
df.head(1)


,아파트_unique_key,시,구,동,시군구,번지,본번,아파트명,건축년도,도로명,좌표Y,좌표X
0,강서구_방화동_546-1_방화3차우림필유_2005,서울특별시,강서구,방화동,서울특별시 강서구 방화동,546-1,546.0,방화3차우림필유,2005,양천로14길 118,37.573351,126.798318


In [19]:
for index, row in df[df['좌표X'].isnull()].iterrows():
    # print(index, row)
    key_query = row['시군구'] + ' ' + row['번지']
    print('key_query' , key_query)
    findRow = 보완할대상keymap[보완할대상keymap['search_key2'] == key_query]
    if findRow.shape[0] > 0:
        x = findRow['좌표X'].values[0]
        y = findRow['좌표Y'].values[0]
        df.loc[index, '좌표X'] = x
        df.loc[index, '좌표Y'] = y
        print('found', y, x)
    # try:
    #     x = xy_df[xy_df['search_key1'] == key_query]['좌표X'].values[0]
    #     y = xy_df[xy_df['search_key1'] == key_query]['좌표Y'].values[0]
    #     unique.loc[index, '좌표X'] = x
    #     unique.loc[index, '좌표Y'] = y
    # except:
    #     print('err', index, row['아파트명'])
    #     continue
    # if pd.isnull(row['좌표X']):
    #     df.at[index, '좌표X'] = df.loc[index-1, '좌표X']
    #     df.at[index, '좌표Y'] = df.loc[index-1, '좌표Y']

key_query 서울특별시 강남구 삼성동 19-4
found 37.517439 127.05007
key_query 서울특별시 강남구 역삼동 712
found 37.501076 127.045346
key_query 서울특별시 강남구 일원동 690-1
found 37.490587 127.079012
key_query 서울특별시 강동구 상일동 134
found 37.5517184559273 127.164857017437
key_query 서울특별시 강동구 상일동 124
found 37.552454 127.170231
key_query 서울특별시 강서구 방화동 201-1
found 37.56886 126.822189
key_query 서울특별시 강서구 방화동 888
found 37.572315 126.818802
key_query 서울특별시 강서구 방화동 207-5
found 37.568068 126.822382
key_query 서울특별시 강서구 방화동 883
found 37.572288 126.81869
key_query 서울특별시 강서구 방화동 892
found 37.571484 126.817525
key_query 서울특별시 강서구 방화동 897
found 37.571237 126.822099
key_query 서울특별시 강서구 방화동 884
found 37.571374 126.820616
key_query 서울특별시 강서구 방화동 168-38
found 37.573104 126.818361
key_query 서울특별시 강서구 방화동 168-114
found 37.573104 126.818361
key_query 서울특별시 강서구 염창동 275-5
found 37.549998 126.867337
key_query 서울특별시 관악구 신림동 1644
found 37.483708310871 126.905433298638
key_query 서울특별시 광진구 자양동 658-14
found 37.5328318624839 127.084626172897
key_query 

In [20]:
print(df['좌표X'].isnull().sum())
print(df['좌표Y'].isnull().sum())

3
3


In [21]:
df[df['좌표X'].isnull()]

,아파트_unique_key,시,구,동,시군구,번지,본번,아파트명,건축년도,도로명,좌표Y,좌표X
3534,동작구_상도동_324_건영_1997,서울특별시,동작구,상도동,서울특별시 동작구 상도동,324,324.0,건영,1997,만양로 26,NaN,NaN
3598,동작구_신대방동_729-24_캐릭터그린빌_2000,서울특별시,동작구,신대방동,서울특별시 동작구 신대방동,729-24,729.0,캐릭터그린빌,2000,보라매로5가길 7,NaN,NaN
8744,마포구_창전동_27-19_마포웨스트리버태영데시앙_2019,서울특별시,마포구,창전동,서울특별시 마포구 창전동,27-19,27.0,마포웨스트리버태영데시앙,2019,서강로3길 32,NaN,NaN


In [22]:
# 3534,동작구_상도동_324_건영_1997,서울특별시 동작구 상도동,324,건영, 37.507857, 126.949298
# 3598,동작구_신대방동_729-24_캐릭터그린빌_2000,서울특별시 동작구 신대방동 729-24,캐릭터그린빌,37.491812, 126.925066
# 8744,마포구_창전동_27-19_마포웨스트리버태영데시앙_2019,서울특별시 마포구 창전동 27-19,마포웨스트리버태영데시앙,37.549839, 126.930509
df.loc[3534, '좌표X'] = 126.949298
df.loc[3534, '좌표Y'] = 37.507857

df.loc[3598, '좌표X'] = 126.925066
df.loc[3598, '좌표Y'] = 37.491812

df.loc[8744, '좌표X'] = 126.930509
df.loc[8744, '좌표Y'] = 37.549839

In [23]:
df[df['좌표X'].isnull()]

,아파트_unique_key,시,구,동,시군구,번지,본번,아파트명,건축년도,도로명,좌표Y,좌표X


In [24]:
df.to_csv('../data/unique_apt_group_xy_v4_최종.csv', index=False)